In [1]:
import sys
import os

# Workaround to make packages work in both Jupyter notebook and Python
MODULE_ROOT_NAME = "AgeEstimator"
MODULE_PATHS = [
    os.path.abspath(os.path.join('..')),
    os.path.abspath(os.path.join('../..')),
    os.path.abspath(os.path.join('../../..'))
]
MODULE_PATHS = list(
    filter(lambda x: x.endswith(MODULE_ROOT_NAME), MODULE_PATHS))
MODULE_PATH = MODULE_PATHS[0] if len(MODULE_PATHS) == 1 else ""
if MODULE_PATH not in sys.path:
    sys.path.append(MODULE_PATH)
    
from server.data.dataset import DataLoader
from server.models.cnn.model import IMAGE_SIZE, IMAGE_SHAPE, CURR_DIR

Using TensorFlow backend.
/home/daniel/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pandas/core/series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [2]:
import numpy as np
import cv2
import shutil
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from multiprocessing import cpu_count
from threading import Thread
from PIL import Image
from functools import partial
import queue

In [3]:
# Shared model to create bottleneck features

model = VGGFace(model="resnet50", include_top=False,
                   input_shape=IMAGE_SHAPE)

dl = DataLoader()
DATASET_DIR = os.path.join(dl.train_dir, "..", "..", "dataset")
x_train_all, y_train_all = dl.load_train()
x_test_all, y_test_all   = dl.load_test()

detector = MTCNN()

WORKERS = cpu_count() - 1
THREADS_PER_CORE = 2

In [17]:
def extract_face(filename="", input_image=None, img_size=IMAGE_SIZE):
    r"""Extract a single face from a given photograph"""

    img = input_image if input_image else plt.imread(filename)
    
    # Create the detector, using default weights

    detection = detector.detect_faces(img)
    
    assert len(detection) > 0, "Probably sth wrong with this image %s" % filename

    # Extract the bounding box from the first face
    x1, y1, width, height = detection[0]['box']
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    
    # Resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(img_size)
    face_array = np.asarray(image)
    
    return face_array.astype('float32')

In [18]:
def preprocess_image(img):
    r"""Used for single image input"""

    print("Processing image...")
    img = extract_face(input_image=img)
    img = np.expand_dims(img, axis=0)
    
    print("Preprocessing input...")
    processed = preprocess_input(img, version=2)
    
    print("Creating bottleneck features...")
    y_hat = model.predict(processed)
    
    print("Done.")
    return y_hat

In [15]:
# def checkpoint(x, y, prefix="tmp-", suffix="", worker_id=0):
#     np.save(os.path.join(CURR_DIR, "checkpoint",
#                          "%sx-worker%d%s" % (prefix, worker_id, suffix)), x)
#     np.save(os.path.join(CURR_DIR, "checkpoint",
#                          "%sy-worker%d%s" % (prefix, worker_id, suffix)), y)
    
# def load_batch_img(x_train, y_train, x_test, y_test, idx=0, checkpoint_limit=4000):
#     print("[Worker%d]: Processing training images..." % idx)
#     x_train_labels   = []
#     x_train_features = []
#     x_train_labels_crash = []
#     x_train_names_crash  = []
#     for i, fname in enumerate(x_train):
#         try:
#             feature = extract_face(fname)
#         except AssertionError as e:
#             print(">>>> [Worker%d]: %s" % (idx, str(e)))
#             x_train_labels_crash.append(y_train[i])
#             x_train_names_crash.append(fname)
#             continue
#         except Exception as e:
#             print(">>>> [Worker%d]: Exception: %s" % (idx, str(e)))
#             print(fname)
#             x_train_labels_crash.append(y_train[i])
#             x_train_names_crash.append(fname)
#             continue
#         x_train_labels.append(y_train[i])
#         x_train_features.append(feature)
 
#     print("[Worker%d]: Processing test images..." % idx)
#     x_test_labels   = []
#     x_test_features = []
#     x_test_labels_crash = []
#     x_test_names_crash  = []
#     for i, fname in enumerate(x_test):
#         try:
#             feature = extract_face(fname)
#         except AssertionError as e:
#             print(">>>> [Worker%d]: %s" % (idx, str(e)))
#             x_test_labels_crash.append(y_train[i])
#             x_test_names_crash.append(fname)
#             continue
#         except Exception as e:
#             print(">>>> [Worker%d]: Exception: %s" % (idx, str(e)))
#             print(fname)
#             x_test_labels_crash.append(y_train[i])
#             x_test_names_crash.append(fname)
#             continue
#         x_test_labels.append(y_test[i])
#         x_test_features.append(feature)
  
#     crashed = (x_train_labels_crash, x_train_names_crash, x_test_labels, x_test_names_crash)
#     return crashed
     
    
def preprocess_dataset(idx=0, total_workers=1):
    r"""Single thread preprocessing task"""
    
    train_limit = len(x_train_all) // total_workers
    test_limit  = len(x_test_all) // total_workers
    
    x_train = x_train_all[idx * train_limit : (idx + 1) * train_limit]
    y_train = y_train_all[idx * train_limit : (idx + 1) * train_limit]
    x_test  = x_test_all[idx * test_limit : (idx + 1) * test_limit]
    y_test  = y_test_all[idx * test_limit : (idx + 1) * test_limit]

    print("[Worker%d]: Processing training images..." % idx)
    x_train_labels   = []
    x_train_features = []
    x_train_crashed = []
    for i, fname in enumerate(x_train):
        try:
            feature = extract_face(fname)
        except AssertionError as e:
            print(">>>> [Worker%d]: %s" % (idx, str(e)))
            x_train_crashed.append((fname, y_train[i]))
            continue
        except Exception as e:
            print(">>>> [Worker%d]: Exception: %s" % (idx, str(e)))
            print(fname)
            x_train_crashed.append((fname, y_train[i]))
            continue
        x_train_labels.append(y_train[i])
        x_train_features.append(feature)
 
    print("[Worker%d]: Processing test images..." % idx)
    x_test_labels   = []
    x_test_features = []
    x_test_crashed = []
    for i, fname in enumerate(x_test):
        try:
            feature = extract_face(fname)
        except AssertionError as e:
            print(">>>> [Worker%d]: %s" % (idx, str(e)))
            x_test_crashed.append((fname, y_train[i]))
            continue
        except Exception as e:
            print(">>>> [Worker%d]: Exception: %s" % (idx, str(e)))
            print(fname)
            x_test_crashed.append((fname, y_train[i]))
            continue
        x_test_labels.append(y_test[i])
        x_test_features.append(feature)
  
    crashed = (x_train_crashed, x_test_crashed)
    
    print("[Worker%d]: Preprocessing inputs..." % idx)
    x_train_processed = preprocess_input(x_train_features, version=2)
    x_test_processed  = preprocess_input(x_test_features, version=2)
    
    print("[Worker%d]: Creating bottleneck features..." % idx)
    p_train = model.predict(x_train_processed)
    p_test  = model.predict(x_test_processed)
    
    try:
        print("[Worker%d]: Removing dataset dir..." % idx)
        shutil.rmtree(DATASET_DIR)
    except:
        pass
    
    print("[Worker%d]: Saving features..." % idx)
    np.save(os.path.join(CURR_DIR, "features-train-worker%d" % idx), p_train)
    np.save(os.path.join(CURR_DIR, "features-test-worker%d" % idx), p_test)
    np.save(os.path.join(CURR_DIR, "labels-train-worker%d" % idx), x_train_labels)
    np.save(os.path.join(CURR_DIR, "labels-test-worker%d" % idx), x_test_labels)
    
#     print("[Worker%d]: Done." % idx)
#     crashed = (x_train_labels_crash, x_train_names_crash, x_test_labels, x_test_names_crash)
#     return p_train, p_test, x_train_labels, x_test_labels, crashed, idx

In [7]:
# def preprocess_dataset_parallel():  
#     """
#     Multithread conflicts MTCNN.
    
#     Change its src code (network/factory.py) from:
#         from keras.layers import Input, Dense, Conv2D, MaxPooling2D, PReLU, Flatten, Softmax
#         from keras.models import Model
#     into
#         from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, PReLU, Flatten, Softmax
#         from tensorflow.keras.models import Model
#     """

#     threads_count = WORKERS * THREADS_PER_CORE
#     print("Using %d threads." % threads_count)
    
#     q = queue.Queue()  # For putting and getting results of thread
#     do_preprocess = lambda idx, total, q: q.put(preprocess_dataset(idx, total))
#     threads = [Thread(
#         target=do_preprocess,
#         args=(idx, threads_count, q)
#     ) for idx in range(threads_count)]
    
#     for t in threads: t.start()
#     for t in threads: t.join()
        
#     res = []
#     for t in threads: res.append(q.get())
        
#     return res

In [5]:
# res = preprocess_dataset_parallel()

In [21]:
# Let's do single thread only :/
preprocess_dataset(0, 1)

[Worker0]: Processing training images...
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/57_Jim_Jarmusch_0013.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_20617.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/49_Ricky_Gervais_0015.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/67_utk_14620.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_22319.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_22626.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/35_utk_21319.jpg
>>>> [Worker0]: Proba

>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/61_utk_11449.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/26_Mary-Kate_Olsen_0008.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/54_utk_20551.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/20_utk_14745.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/38_utk_08547.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/42_Thomas_Kretschmann_0004.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/45_Mark_Sheppard_0018.jpg
>>>> [Worker0]: Probably sth wrong with th

>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/22_utk_19170.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/27_James_McAvoy_0008.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/3_utk_08773.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_13662.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/25_utk_13617.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_05280.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/32_utk_03710.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tch

>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/88_utk_05535.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_12770.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_10012.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_16850.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/37_utk_05068.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/26_utk_20934.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/36_Jason_Lee_0011.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchian

>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/26_Tammin_Sursok_0004.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/1_utk_20569.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/3_utk_06759.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/56_Spike_Lee_0011.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/51_Rowan_Atkinson_0003.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/27_Lady_Gaga_0009.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/88_utk_12974.jpg
>>>> [Worker0]: Probably sth wrong with this imag

>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/26_Ashley_Olsen_0007.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/16_utk_10089.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/26_utk_17815.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/25_utk_14724.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/58_Jamey_Sheridan_0011.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/3_utk_14857.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/train/2_utk_13500.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/use

>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/test/46_Arnold_Vosloo_0004.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/test/1_utk_14963.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/test/45_Amy_Sedaris_0014.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/test/14_utk_19767.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/test/110_utk_04520.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/test/22_Lady_Gaga_0009.jpg
>>>> [Worker0]: Probably sth wrong with this image /WAVE/users/unix/tchiang/AgeEstimator/server/data/dataset/test/31_Lauren_Cohan_0007.jpg
>>>> [Worker0]: Probably sth wrong with this image /WA